In [1]:
#importing libraries
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

In [3]:
pip install selenium

In [1]:
#open the browser

driver=webdriver.Chrome()

#load the webpage

driver.get("https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile")
#driver.get("https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile")

time.sleep(3)

driver.maximize_window()

NameError: name 'webdriver' is not defined

In [21]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
import time

#implicit wait
wait = WebDriverWait(driver, 20)

def Kerala_link_route(path):   
    LINKS_KERALA = []
    ROUTE_KERALA = []

    # Retrieve the route links and route names
    for i in range(1, 4):  # Adjust the range based on the actual number of pages, e.g., 4
        paths = driver.find_elements(By.XPATH, path)
        
        # Retrieve links
        for links in paths:
            d = links.get_attribute("href")
            LINKS_KERALA.append(d)
        
        # Retrieve names of the routes
        for route in paths:
            ROUTE_KERALA.append(route.text)

        try:
            # Wait for the pagination element to be present
            pagination = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@class="DC_117_paginationTable"]')))
            
            # Check if next button for the next page exists
            next_button_xpath = f'//div[@class="DC_117_pageTabs " and text()={i+1}]'
            if len(driver.find_elements(By.XPATH, next_button_xpath)) > 0:
                next_button = driver.find_element(By.XPATH, next_button_xpath)
                
                # Scroll into view and click the next button using JavaScript
                driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
                time.sleep(2)  # Ensure some time for scrolling

                # Explicitly wait for the next button to be clickable
                wait.until(EC.element_to_be_clickable(next_button))
                driver.execute_script("arguments[0].click();", next_button)  # Click using JavaScript

            else:
                print(f"No more pages to paginate at step {i}")
                break  # No more pages, exit the loop

        except (NoSuchElementException, ElementClickInterceptedException) as e:
            print(f"Encountered an issue at step {i}: {e}")
            break
        
    return LINKS_KERALA, ROUTE_KERALA

# Calling the function to get links and routes
LINKS_KERALA, ROUTE_KERALA = Kerala_link_route("//a[@class='route']")


In [3]:
df_k=pd.DataFrame({"Route_name":ROUTE_KERALA,"Route_link":LINKS_KERALA})
df_k

NameError: name 'pd' is not defined

In [24]:
df = df_k

In [4]:

#retrive the bus details
driver_k = webdriver.Chrome()
Bus_names_k = []
Bus_types_k = []
Start_Time_k = []
End_Time_k = []
Ratings_k = []
Total_Duration_k = []
Prices_k = []
Seats_Available_k = []
Route_names = []
Route_links = []

for i,r in df.iterrows():
    link=r["Route_link"]
    routes=r["Route_name"]

# Loop through each link
    driver_k.get(link)
    time.sleep(2)  

    # Click on elements to reveal bus details
    elements = driver_k.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)
        
    # click elements to views bus
    try:
        clicks = driver_k.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except:
        continue  
    time.sleep(2)
    
    scrolling = True
    while scrolling:
        old_page_source = driver_k.page_source
        
        # Use ActionChains to perform a PAGE_DOWN
        ActionChains(driver_k).send_keys(Keys.PAGE_DOWN).perform()
        
        time.sleep(5)  
        
        new_page_source = driver_k.page_source
        
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details
    bus_name = driver_k.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_k.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_k.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_k.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_k.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    try:
        rating = driver_k.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    price = driver_k.find_elements(By.XPATH, '//*[@class="fare d-block"]')
    seats = driver_k.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Append data to respective lists
    for bus in bus_name:
        Bus_names_k.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_k.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Start_Time_k.append(start_time_elem.text)
    for end_time_elem in end_time:
        End_Time_k.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_k.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_k.append(ratings.text)
    for price_elem in price:
        Prices_k.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_k.append(seats_elem.text)
        
print("Successfully Completed")

NameError: name 'webdriver' is not defined

In [5]:
# from list to convert data frame
data = {
    'Bus_name': Bus_names_k,
    'Bus_type': Bus_types_k,
    'Start_time': Start_Time_k,
    'End_time': End_Time_k,
    'Total_duration': Total_Duration_k,
    'Price': Prices_k,
    "Seats_Available":Seats_Available_k,
    "Ratings":Ratings_k,
    'Route_link': Route_links,
    'Route_name': Route_names
}
df1 = pd.DataFrame(data)

NameError: name 'Bus_names_k' is not defined

In [ ]:

df1 